In [1]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
es.info()

ObjectApiResponse({'name': '381f7333ac12', 'cluster_name': 'docker-cluster', 'cluster_uuid': '3SWSfK9cRq-AwvN2yYbsWQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [2]:
#1 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73'

In [3]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [4]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()
documents = []

for course in documents_raw:
    t = {}
    for doc in course['documents']:
        t['course'] = course['course']
        t['text'] = doc['text']
        documents.append(t)

In [5]:
from tqdm.auto import tqdm
index_name = "course-questions"

for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [6]:
#2 index

In [7]:
query = "How do I execute a command in a running docker container?"

In [8]:
search_query = {
        "size": 10,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
}

In [9]:
es.info()

ObjectApiResponse({'name': '381f7333ac12', 'cluster_name': 'docker-cluster', 'cluster_uuid': '3SWSfK9cRq-AwvN2yYbsWQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [10]:
response = es.search(index=index_name, body=search_query)

In [11]:
response['hits']['max_score']

84.26358

In [12]:
#3 84.26358

In [13]:
search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
}

In [14]:
response = es.search(index=index_name, body=search_query)

In [15]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': '3NjTNJABn0mP0pcGj_9a',
  '_score': 84.26358,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'kNneNJABn0mP0pcG_gOX',
  '_score': 84.26358,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> 

In [16]:
#4 'How do I debug a docker container?'

In [17]:
q = response['hits']['hits'][0]['_source']
q['question']


'How do I debug a docker container?'

In [18]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

context = ""

for doc in response['hits']['hits']:
    source = doc['_source']
    context = context + f"section: {source['section']}\nquestion: {source['question']}\nanswer: {source['text']}\n\n"

prompt = prompt_template.format(question=query, context=context).strip()

In [19]:
len(prompt)

1497

In [ ]:
#5 1497

In [21]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [23]:
len(encoding.encode(prompt))

332

In [ ]:
#6 322